In [84]:
import polars as pl
import pandas as pd

## Abro sitios y reviews

In [149]:
pl_sitios = pl.read_parquet('parquet/sitios_FL.parquet')
pl_reviews = pl.read_parquet('parquet/review_florida.parquet')
pl_reviews.shape

(2850000, 9)

## Creo un dataframe a partir de un diccionario que est compuesto por las combinaciones del
## nombre del restaurant + relative_results
## Se llama pl_name_review

In [150]:
i = 0
cantidad = pl_sitios.shape[0]
dicc = {'name':[],'gmap_id':[]}
for i in range(0,cantidad-1):
    if pl_sitios[i,'relative_results']:
        lista = pl_sitios[i,'relative_results'].split(',')
        for elem in lista:
            dicc['name'].append(pl_sitios[i,'name'])
            dicc['gmap_id'].append(elem)
    i = i + 1
pl_review_FL = pl.DataFrame(dicc)    
pl_review_FL.shape

(13788, 2)

## Hago un join entre 
## pl_name_review y pl_reviews

In [151]:
pl_review_FL = pl_review_FL.join(pl_reviews.select(['gmap_id','name','rating','text']), on='gmap_id')
pl_review_FL.shape

(285444, 5)

## Elimino duplicados

In [152]:
pl_review_FL = pl_review_FL.groupby(['name','gmap_id']).agg([
    pl.col('rating').mean(),
    pl.col('text')
]).sort('name')
pl_review_FL = pl_review_FL.with_columns([
    pl.col('rating').apply(lambda x: round(x,2))
])
pl_review_FL.shape

(930, 4)

In [153]:
pl_review_FL.columns

['name', 'gmap_id', 'rating', 'text']

## Elimino gmap_id, ya no lo necesito

In [154]:
pl_review_FL = pl_review_FL.select(pl.exclude('gmap_id'))

In [155]:
pl_review_FL.head()

name,rating,text
str,f64,list[str]
"""33 Below Icecr...",4.0,"[""Nice place, good ice cream. They could do better with the amount of serving. They serve more ice cream in Argentina."", ""Nice place, good ice cream. They could do better with the amount of serving. They serve more ice cream in Argentina."", ... null]"
"""33 Below Icecr...",3.69,"[""Not open during posted hours"", ""Not open during posted hours"", ... null]"
"""3Natives""",4.53,"[""Stopped in for lunch. Ordered what my friend ordered...the Curry Chicken Wrap. Clean open space. Well lit. Fast and friendly service. A health fanatic’s haven I would guess. Never had canned water! The wrap was really good as well as the dressing that comes with it. I’ll be back to try a bowl and smoothie. 😋"", ""I always feel great after I stop in for an acai bowl. Fresh, healthy ingredients are perfect for a light lunch or heavy snack. Very easy to order online for a quick pick-up. On the other hand, they have enough space that eating in works well."", ... null]"
"""3Natives""",4.2,"[""I asked questions about yogurt based smoothies. There were 2 attendants. The younger one was fine but cut off from helping me by the older, more abrupt one whose eyes literally bulged out in impatience and incredulity when asked simple questions about the smoothies. No desire or ability to explain or take time with a new customer. Will definitely not be returning here. Time: 12:45 pm on 06/04/21."", ""A tastey option for smoothies that is also healthy! You can also add additional fresh fruits to the base to satisfy your taste buds. Wondrous treats that wont break the bank or the belt. You get my thumbs up."", ... null]"
"""3Natives""",4.32,"[""They have such yummy acai bowls and light food options! I definitely recommend!"", ""Always delicious . Açaí bowls are always fresh and good."", ... null]"


## Salvo como rest_review_FL(parquet y csv)

In [156]:
pl_review_FL.write_parquet('parquet/review_FL.parquet')
#pl_review_FL.write_csv('csv/review_FL.csv')
pl_review_FL.write_parquet('PG_Google_Maps/Ornaldo/review_FL.parquet')